<a href="https://colab.research.google.com/github/pablosierrafernandez/FarmacoDB/blob/main/model_pipeline/finalist_models_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo
Proporcionar un modelo para la predicción de la **Vida Media** utilizando descriptores moleculares.
Usted podrá usar el parámetro farmacocinético que desee. Simplemente reemplace el `.csv` correspondiente al parámetro.

La finalidad principal consiste en ofrecer un pipeline para poder aplicar a la estimación de otros parámetros farmacocinéticos contemplados en la base de datos, facilitando así una herramienta valiosa en el campo de la farmacocinética y la farmacología personalizada.


---



# Índice:
1.   Importación de bibliotecas
2.   Limpieza de datos
3.   Construcción de modelos
4.   Modelo a producción
---




# 1. Importación de bibliotecas

In [ ]:

from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

## 1.1 Carga del conjunto de datos

In [ ]:
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/halflife.csv')  # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo


# 2. Limpieza de datos


Se han realizado diversas operaciones de limpieza de datos, incluyendo eliminación de valores faltantes, columnas no numéricas, y valores atípicos.

## 2.1. Limpieza

In [ ]:
# Identifica las columnas no numéricas
non_numeric_columns = []
for column in data.columns:
    if data[column].dtype == 'object':
        non_numeric_columns.append(column)

# Elimina las columnas no numéricas
data = data.drop(columns=non_numeric_columns)
# Muestra las columnas no numéricas
print("Columnas no numéricas:")
print(non_numeric_columns)

# Número de filas antes de eliminar
num_filas_antes = len(data)

# Elimina las filas con valores faltantes
data.dropna(inplace=True)

# Número de filas después de eliminar
num_filas_despues = len(data)

# Muestra los resultados
print("Número de filas antes de eliminar:", num_filas_antes)
print("Número de filas después de eliminar:", num_filas_despues)
# Seleccionar columnas con valores booleanos
bool_columns = data.select_dtypes(include=['bool']).columns
print(bool_columns)
# Eliminar las columnas con valores booleanos
data = data.drop(columns=bool_columns)
# Reemplazar los valores NaN con 0.0
data = data.fillna(0.0)

# Eliminar columnas que contienen solo valores 0.0
data = data.loc[:, (data != 0.0).any(axis=0)]

## 2.2. Outliers (opcional)

In [ ]:
x = data['median'].values
ax = sns.boxplot(x)

print('The meadian is: ', data['median'].median())

In [ ]:
# Notas:
# La línea representa la mediana
# El cuadro en el medio muestra el comienzo del Q1 (percentil 25) y el final del Q3 (percentil 75)
# Las líneas (izquierda - derecha) muestran el cuartil mínimo y el cuartil máximo
# Los puntos en la derecha son "valores atípicos"


Si nota que quedan muchos valores atípicos aplique el siguiente codigo para eliminar outliers.

In [ ]:

# Calcula el primer y tercer cuartil
Q1 = data['median'].quantile(0.25)
Q3 = data['median'].quantile(0.75)

# Calcula el rango intercuartílico (IQR)
IQR = Q3 - Q1

# Define los límites superior e inferior para identificar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtra los datos para eliminar los outliers
data = data[(data['median'] >= lower_bound) & (data['median'] <= upper_bound)]


## 2.3. Selección de características

Puede aplicar otro método de selección de características que considere. En este caso se aplica 'Backward Elimination'

In [ ]:


cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
X = data[selected_features_BE]

y = data["median"]  # Columna objetivo




## 2.4. Test y Train

División del conjunto de datos en entrenamiento y prueba

In [ ]:


X = X.values.astype(np.float64)
y = y.values.astype(np.float64)




X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)





## 2.5. Normalización y estandarización

Elija el metodo de normalización y/o estandarización deseado a aplicar a la salida y/o entrada de los datos del modelo.

In [ ]:

input_scaler = StandardScaler() #MinMaxScaler(), StandardScaler(), None
output_scaler = StandardScaler()#MinMaxScaler(), StandardScaler(), None

if input_scaler is not None:
    input_scaler.fit(X_train)
    X_train = input_scaler.transform(X_train)
    X_test = input_scaler.transform(X_test)
if output_scaler is not None:
    y_train = y_train.reshape(len(y_train), 1)
    y_test = y_test.reshape(len(y_test), 1)
    output_scaler.fit(y_train)
    y_train = output_scaler.transform(y_train)
    y_test = output_scaler.transform(y_test)

# 3. Construcción de modelos

A continuación, se procede a la construcción y entrenamiento de modelos predictivos.

## 3.1. Red Neuronal Artificial (ANN)

### 3.1.1. Entrenamiento

In [ ]:
# Definición del modelo
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

In [ ]:
# Entrenamiento del modelo
model = build_model()
model.fit(X_train, y_train, epochs=1000, verbose=1)


### 3.1.2. Visualización

In [ ]:
y_train = output_scaler.inverse_transform(y_train)
y_test = output_scaler.inverse_transform(y_test)
y_pred = output_scaler.inverse_transform(model.predict(X_test))
trainpreds = output_scaler.inverse_transform(model.predict(X_train))
X_train=output_scaler.inverse_transform(X_train)
X_test=output_scaler.inverse_transform(X_test)
model.evaluate(X_train,y_train)
model.evaluate(X_test,y_test)
print("-------------")
print("MAE - TEST:",metrics.mean_absolute_error(y_test,y_pred))
print("MAE - TRAIN:",metrics.mean_absolute_error(y_train, trainpreds))
print("MSE:",metrics.mean_squared_error(y_test,y_pred))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("-------------")
y_real = np.array(y_test[:50])
y_predd = np.array(y_pred[:50])
indices = np.arange(len(y_real))
plt.figure(figsize=(8, 4))
plt.plot(indices, y_real, marker='o', label='Real', linestyle='-')
plt.plot(indices, y_predd, marker='o', label='Predicción', linestyle='--')
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.title('Comparación entre Valores Reales y Predicciones')
plt.legend()
plt.show()
y_real = np.array(y_test[:5])
y_predd = np.array(y_pred[:5])
indices = np.arange(len(y_real))
plt.figure(figsize=(8, 4))
plt.plot(indices, y_real, marker='o', label='Real', linestyle='-')
plt.plot(indices, y_predd, marker='o', label='Predicción', linestyle='--')
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.title('Comparación entre Valores Reales y Predicciones')
plt.legend()
plt.show()
# scatterplot of actual vs. pred
fig, axes = plt.subplots(1,2)
axes[0].scatter(x=y_train, y=model.predict(X_train))
axes[0].set_xlabel("Actual", fontsize=10)
axes[0].set_ylabel("Predicted",  fontsize=10)
axes[0].set_title("Training")
x = np.linspace(*axes[0].get_xlim())
axes[0].plot(x, x, color='red')
axes[1].scatter(x=y_test, y=model.predict(X_test))
axes[1].set_xlabel("Actual", fontsize=10)
axes[1].set_ylabel("Predicted",  fontsize=10)
axes[1].set_title("Validation")
x = np.linspace(*axes[1].get_xlim())
axes[1].plot(x, x, color='red')
fig.tight_layout()
plt.show()

## 3.2 MLJAR

In [ ]:
!pip install fastapi

In [ ]:
!pip install kaleido

In [ ]:
!pip install uvicorn

In [ ]:
!pip install python-multipart

In [ ]:
!pip install mljar-supervised

In [ ]:
!pip install pandas==2.0.0

In [ ]:
y_train = y_train.reshape(-1)  # Convierte en un vector 1D

### 3.2.1 Entrenamiento

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from supervised.automl import AutoML # mljar-supervised



automl = AutoML(mode="Compete" , total_time_limit=900)
automl.fit(X_train, y_train)


predictions = automl.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, predictions))
model = automl
y_train = y_train.reshape(-1, 1)

### 3.2.2 Visualización

In [ ]:
y_train = output_scaler.inverse_transform(y_train)
y_test = output_scaler.inverse_transform(y_test)
y_pred = output_scaler.inverse_transform(model.predict(X_test).reshape(-1, 1))
trainpreds = output_scaler.inverse_transform(model.predict(X_train).reshape(-1, 1))
X_train=output_scaler.inverse_transform(X_train)
X_test=output_scaler.inverse_transform(X_test)

from sklearn import metrics
print("MAE - TEST:",metrics.mean_absolute_error(y_test,y_pred))
print("MAE - TRAIN:",metrics.mean_absolute_error(y_train, trainpreds))
print("-------------")
print ("MSE:",metrics.mean_squared_error(y_test,y_pred))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
# Valores reales
y_real = np.array(y_test[:50])

# Predicciones
y_predd = np.array(y_pred[:50])
# Crear un array con los índices para etiquetar los puntos en el gráfico
indices = np.arange(len(y_real))

# Crear una figura y ejes
plt.figure(figsize=(8, 4))
plt.plot(indices, y_real, marker='o', label='Real', linestyle='-')
plt.plot(indices, y_predd, marker='o', label='Predicción', linestyle='--')

# Etiquetas y título del gráfico
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.title('Comparación entre Valores Reales y Predicciones')

# Mostrar una leyenda
plt.legend()

# Mostrar el gráfico
plt.show()
# scatterplot of actual vs. pred

fig, axes = plt.subplots(1,2)


axes[0].scatter(x=y_train, y=model.predict(X_train))
axes[0].set_xlabel("Actual", fontsize=10)
axes[0].set_ylabel("Predicted",  fontsize=10)
axes[0].set_title("Training")

x = np.linspace(*axes[0].get_xlim())
axes[0].plot(x, x, color='red')

axes[1].scatter(x=y_test, y=model.predict(X_test))
axes[1].set_xlabel("Actual", fontsize=10)
axes[1].set_ylabel("Predicted",  fontsize=10)
axes[1].set_title("Validation")

x = np.linspace(*axes[1].get_xlim())
axes[1].plot(x, x, color='red')


fig.tight_layout()

plt.show()
import matplotlib.pyplot as plt

a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred)
plt.xlabel('X')
plt.ylabel('y')
_=plt.plot(y_test,y_test,color='orange')

# 4. Modelo a producción

## 4.1 Guardar modelo

In [ ]:

model.save("mi_modelo.h5")


## 4.2 Cargar modelo

In [ ]:
from tensorflow.keras.models import load_model

modelo_cargado = load_model("mi_modelo.h5")
